In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
import math
import os

class DebugLungeCounter:
    def __init__(self, model_name='yolo11s-pose.pt'):
        print(f"Loading {model_name}...")
        self.model = YOLO(model_name)
        self.lunge_count = 0
        self.lunge_stage = "up"
        self.confidence_threshold = 0.3
        self.knee_down_threshold = 135
        self.knee_up_threshold = 155
        self.knee_angles_history = deque(maxlen=5)
        self.frame_count = 0
        self.pose_detected_count = 0
        self.knee_detected_count = 0

    def detect_pose_yolo11(self, frame):
        results = self.model(frame)
        if results[0].keypoints is not None and len(results[0].keypoints.xy) > 0:
            keypoints = results[0].keypoints.xy[0].cpu().numpy()
            confidences = results[0].keypoints.conf[0].cpu().numpy() if results[0].keypoints.conf is not None else None
            self.pose_detected_count += 1
            return keypoints, confidences, results[0]
        return None, None, results[0]

    def calculate_angle(self, point1, point2, point3):
        try:
            a, b, c = np.array(point1), np.array(point2), np.array(point3)
            ba, bc = a - b, c - b
            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
            angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
            return np.degrees(angle)
        except:
            return 180

    def get_knee_angles(self, keypoints, confidences):
        if keypoints is None or len(keypoints) < 17:
            return None, None, None, None
        left_hip, left_knee, left_ankle = keypoints[11], keypoints[13], keypoints[15]
        right_hip, right_knee, right_ankle = keypoints[12], keypoints[14], keypoints[16]
        left_conf = min(confidences[11], confidences[13], confidences[15]) if confidences is not None else 1.0
        right_conf = min(confidences[12], confidences[14], confidences[16]) if confidences is not None else 1.0
        left_angle = self.calculate_angle(left_hip, left_knee, left_ankle) if left_conf > self.confidence_threshold else None
        right_angle = self.calculate_angle(right_hip, right_knee, right_ankle) if right_conf > self.confidence_threshold else None
        self.knee_detected_count += int(left_angle is not None) + int(right_angle is not None)
        return left_angle, right_angle, left_conf, right_conf

    def update_lunge_count(self, left_angle, right_angle):
        working_angle = left_angle if right_angle is None else right_angle if left_angle is None else min(left_angle, right_angle)
        working_leg = "left" if working_angle == left_angle else "right"
        if working_angle is None:
            return False, None
        self.knee_angles_history.append(working_angle)
        smoothed_angle = np.mean(self.knee_angles_history) if len(self.knee_angles_history) >= 3 else working_angle
        lunge_detected = False
        if smoothed_angle < self.knee_down_threshold and self.lunge_stage == "up":
            self.lunge_stage = "down"
        elif smoothed_angle > self.knee_up_threshold and self.lunge_stage == "down":
            self.lunge_stage = "up"
            self.lunge_count += 1
            lunge_detected = True
        return lunge_detected, working_leg

    def get_count(self):
        return self.lunge_count

    def get_stage(self):
        return self.lunge_stage

def process_debug_lunge_video(video_path, model_name='yolo11s-pose.pt', output_path="lunge_output2.mp4"):
    counter = DebugLungeCounter(model_name=model_name)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    milestone_timer = 0

    for frame_idx in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        counter.frame_count += 1
        keypoints, confidences, results = counter.detect_pose_yolo11(frame)
        if keypoints is not None:
            frame = results.plot()
        left_angle, right_angle, _, _ = counter.get_knee_angles(keypoints, confidences)
        lunge_detected, working_leg = counter.update_lunge_count(left_angle, right_angle)

        if lunge_detected:
            overlay = frame.copy()
            cv2.rectangle(overlay, (0, 0), (width, height), (0, 255, 0), -1)
            frame = cv2.addWeighted(frame, 0.8, overlay, 0.2, 0)
            cv2.putText(frame, "LUNGE!", (width//2 - 80, height//2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
            if counter.get_count() % 10 == 0:
                milestone_timer = 30  # show milestone for next 30 frames

        y_offset = 30
        cv2.putText(frame, f"LUNGES: {counter.get_count()}", (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (202, 255, 150), 2)
        y_offset += 25

        # XP counter (10 XP per lunge)
        xp_value = counter.get_count() * 10
        cv2.putText(frame, f"XP: {xp_value}", (width - 150, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 150, 50), 2)

        # Milestone celebration: "You Rock!" banner
        if milestone_timer > 0:
            cv2.putText(frame, "🎉 YOU ROCK! 🎉", (width//2 - 150, height//2 - 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 4)
            milestone_timer -= 1

        out.write(frame)

    cap.release()
    out.release()
    print(f"Output: {output_path}")
    return output_path, counter.get_count()

if __name__ == "__main__":
    VIDEO_DIR = "/Users/shruti.kalaskar/Documents/Northwestern/Spring 2025/Computer Vision/Project/"
    MODEL_NAME = 'yolo11s-pose.pt'
    video_files = ["lunge_reformer_1.mp4", "lunge_reformer_2.mp4"]
    for video_file in video_files:
        input_path = os.path.join(VIDEO_DIR, video_file)
        output_path = os.path.join(VIDEO_DIR, video_file.replace(".mp4", "_output.mp4"))
        if os.path.exists(input_path):
            print(f"\n▶️ Processing video: {input_path}")
            process_debug_lunge_video(video_path=input_path, model_name=MODEL_NAME, output_path=output_path)
        else:
            print(f"File not found: {input_path}")



▶️ Processing video: /Users/shruti.kalaskar/Documents/Northwestern/Spring 2025/Computer Vision/Project/lunge_reformer_1.mp4
Loading yolo11s-pose.pt...

0: 640x384 1 person, 250.8ms
Speed: 9.2ms preprocess, 250.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 166.4ms
Speed: 0.8ms preprocess, 166.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 166.7ms
Speed: 1.1ms preprocess, 166.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 161.3ms
Speed: 1.0ms preprocess, 161.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 181.7ms
Speed: 1.7ms preprocess, 181.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 221.4ms
Speed: 1.0ms preprocess, 221.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 183.1ms
Speed: 0.8ms preprocess, 183.1ms inference, 0.8ms 